#**Importing necessary libraries**


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

#**Importing dataset**


In [2]:
df = pd.read_csv("megaGymDataset.csv")
df.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2918 non-null   int64  
 1   Title       2918 non-null   object 
 2   Desc        1368 non-null   object 
 3   Type        2918 non-null   object 
 4   BodyPart    2918 non-null   object 
 5   Equipment   2918 non-null   object 
 6   Level       2918 non-null   object 
 7   Rating      1031 non-null   float64
 8   RatingDesc  862 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 205.3+ KB


In [4]:
df.describe()

,Unnamed: 0,Rating
count,2918.000000,1031.000000
mean,1458.500000,5.919690
std,842.498368,3.584607
min,0.000000,0.000000
25%,729.250000,3.000000
50%,1458.500000,7.900000
75%,2187.750000,8.700000
max,2917.000000,9.600000


In [89]:
df.columns = df.columns.str.replace('Unnamed: 0', 'index')
df['Equipment'] = df['Equipment'].fillna('None')


#**Getting input from user**

#Importing model

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"  # Example model, choose one that fits your needs
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Instantiate question answering pipeline

In [7]:
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

#Getting answers


In [36]:
context=input("Hello, what are you looking for?");

questions = [
    "What is your level at working out : Beginner or Advanced or Expert?",
    "How many days per week can you go to the gym?",
    "What is your goal : Cut or Bulk or Lean or Strength or Athletic ?",
]

information={}
for question in questions:
    # Get answer from the context using the question answering pipeline
    answer = qa_pipeline(question=question, context=context)
    print(answer['score'])
    while(answer['score']<0.5) :
      x=input("Can you precise "+question)
      answer = qa_pipeline(question=question, context=x)
    information[question] = answer["answer"]
    print(f"Question: {question}")
    print(f"Answer: {answer['answer']} (with confidence score: {answer['score']:.2f})")


Hello, what are you looking for?I have 4 days available this week and I want to cut. I consider myself an intermediate level
0.5616934299468994
Question: What is your level at working out : Beginner or Advanced or Expert?
Answer: intermediate (with confidence score: 0.56)
0.7363378405570984
Question: How many days per week can you go to the gym?
Answer: 4 (with confidence score: 0.74)
0.3747851252555847
Can you precise What is your goal : Cut or Bulk or Lean or Strength or Athletic ?lean
Question: What is your goal : Cut or Bulk or Lean or Strength or Athletic ?
Answer: lean (with confidence score: 0.98)


#Extracting key words

In [44]:
def extract_level(answer):
    tokens = answer.lower().split()
    level_keywords = ["beginner", "intermediate", "advanced", "expert"]

    for token in tokens:
        if token in level_keywords:
            return token.capitalize()
    return None

def extract_goal(answer):
    tokens = answer.lower().split()
    goal_keywords = ["cut", "bulk", "strength", "lean"]
    for token in tokens:
        if token in goal_keywords:
            return token.capitalize()
    return None

def extract_day(answer):
    return int(answer)

In [88]:
first_answer = list(information.values())[0]

second_answer = list(information.values())[1]

third_answer = list(information.values())[2]

level=extract_level(first_answer)
goal=extract_goal(third_answer)
days=extract_day(second_answer)


In [ ]:
level = input("What is your level?")
days = int(input("How many days per week can you go to the gym?"))
goal = input("What is your goal ?") #Cut/Bulk/Lean/Strength/Athletic


What is your level?Beginner
How many days per week can you go to the gym?4
What is your goal ?Cut


In [21]:
home = input("Do you prefer to workout at home?")
if (home=="Yes") :
  equip=input("what equipment do you have?")
  equipments = equip.split(",")
  equipments.append("None")
  print(equipments)

Do you prefer to workout at home?Yes
what equipment do you have?Bands,Barbell,Dumbells
['Bands', 'Barbell', 'Dumbells', 'NaN']


In [46]:
programs={}
Push = [{'Triceps' : 2}, {'Shoulders' : 2} , {'Chest' :3} ]
Pull = [{'Biceps' :2}, {'Forearms' :1}, {'Lats' :1}, {'Middle Back':1}, {'Traps':1}]
Legs = [ {'Abductors' :1} , {'Hamstrings' :2}, {'Quadriceps':2}]
Glutes = [{'Adductors':1}, {'Glutes':1}]
Core = [{'Abdominals' :3}, {'Lower Back':2}]

Upper = Push + Pull
Lower = Legs + Glutes


#**Choosing program based on available days and goal**

In [47]:
if days==2:
    programs.update({'Upper': Upper})
    programs.update({'Legs' : Legs})
elif days == 3:
    programs['Push'] = Push
    programs['Pull'] = Pull
    programs['Legs'] = Legs
elif days==4:
    programs['Push'] = Push
    programs['Pull'] = Pull
    programs['Legs'] = Legs
    programs['Core'] = Core
elif days == 5:
    programs['Chest'] = Push + ['Chest']
    programs['Back'] = Pull + ['Middle Back']
    programs['Shoulder'] = Push + ['Shoulders']
    programs['Legs'] = Legs
    programs['Core'] = Core
print(programs)


{'Push': [{'Triceps': 2}, {'Shoulders': 2}, {'Chest': 3}], 'Pull': [{'Biceps': 2}, {'Forearms': 1}, {'Lats': 1}, {'Middle Back': 1}, {'Traps': 1}], 'Legs': [{'Abductors': 1}, {'Hamstrings': 2}, {'Quadriceps': 2}], 'Core': [{'Abdominals': 3}, {'Lower Back': 2}]}


In [51]:
Types=[]
if (goal=="Cut") :
  Types=[{'Cardio' : 0.5},{'Strength' : 0.5}]
elif(goal=="Lean") :
  Types=[{'Cardio' : 0.25},{'Strength' : 0.75}]
elif(goal=="Bulk") :
  Types=[{'Powerlifting' : 0.5},{'Strength' : 0.5}]
elif(goal=="Strength") :
  Types=[{'strongman' : 0.5},{'Strength' : 0.5}]
elif(goal=="Athletic") :
  Types=[{'Plyometrics' : 0.5},{'Olympic Weightlifting' : 0.5}]
print(Types)
print(goal)

[{'Cardio': 0.25}, {'Strength': 0.75}]
Lean


In [52]:
key1=list(Types[0].keys())[0]
key2=list(Types[1].keys())[0]
coeff1=Types[0][key1]
coeff2=Types[1][key2]

#**Affecting program exercises**

In [91]:
from IPython.display import display
selected_program = pd.DataFrame(columns=['Day','BodyPart', 'Type', 'SelectedExercises'])
if(home=="No"):
  for day in programs:
      for part in programs[day]:
        filtered_exercises = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key1)]
        top_exercises = filtered_exercises.sort_values(by='Rating', ascending=False).head(round(part[next(iter(part))]*coeff1))
        exercise_titles = top_exercises['Title'].tolist()
        count_exos=len(top_exercises)
        if(count_exos>=0):
          selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key1 ,'SelectedExercises': [exercise_titles]})], ignore_index=True)

          filtered_exercises_S = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key2) & (df['Level'] == level)]
          top_exercises_S = filtered_exercises_S.sort_values(by='Rating', ascending=False).head(part[next(iter(part))]-count_exos)
          exercise_titles_S=top_exercises_S['Title'].tolist()
        selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key2, 'SelectedExercises': [exercise_titles_S]})], ignore_index=True)
else :
  for day in programs:
      for part in programs[day]:
        filtered_exercises = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key1) & df['Equipment'].isin(equipments)]
        top_exercises = filtered_exercises.sort_values(by='Rating', ascending=False).head(round(part[next(iter(part))]*coeff1))
        exercise_titles = top_exercises['Title'].tolist()
        count_exos=len(top_exercises)
        if(count_exos>=0):
          selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key1 ,'SelectedExercises': [exercise_titles]})], ignore_index=True)

          filtered_exercises_S = df[(df['BodyPart'] == next(iter(part))) & (df['Type'] == key2) & (df['Level'] == level)]
          top_exercises_S = filtered_exercises_S.sort_values(by='Rating', ascending=False).head(part[next(iter(part))]-count_exos)
          exercise_titles_S=top_exercises_S['Title'].tolist()
        selected_program = pd.concat([selected_program, pd.DataFrame({'Day' : [day] , 'BodyPart': [next(iter(part))], 'Type' : key2, 'SelectedExercises': [exercise_titles_S]})], ignore_index=True)
display(selected_program)


,Day,BodyPart,Type,SelectedExercises
0,Push,Triceps,Cardio,[]
1,Push,Triceps,Strength,"[Triceps dip, Decline EZ-bar skullcrusher]"
2,Push,Shoulders,Cardio,[]
3,Push,Shoulders,Strength,"[Dumbbell front raise to lateral raise, Clean ..."
4,Push,Chest,Cardio,[]
5,Push,Chest,Strength,"[Pushups, Dumbbell Bench Press, Close-grip ben..."
6,Pull,Biceps,Cardio,[]
7,Pull,Biceps,Strength,"[EZ-bar spider curl, Hammer Curls]"
8,Pull,Forearms,Cardio,[]
9,Pull,Forearms,Strength,[Palms-down wrist curl over bench]
